<a href="https://colab.research.google.com/github/aminghafari1/lcaging-fmri/blob/main/task_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
import glob
import os
def substring_after(s,delim):
  return s.partition(delim)[2]
def substring_before(s,delim):
  return s.partition(delim)[0]
import scipy.io as sio
import os
from scipy.stats import zscore
from os import path
import pandas as pd


Mounted at /content/gdrive


In [2]:
base_dir=glob.glob('/content/gdrive/Shareddrives/LC-Aging/'\
  'Older Adult - MRI Study/Data Collection/BAP/BAP data/')
base_dir=base_dir[0]

prefix = 'sub-BAP'
sub_folders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f)) and f.startswith(prefix)]
sub_folders = [base_dir + i for i in sub_folders]
incomplete_directories=[]
print(sub_folders)
for i in range(len(sub_folders)):
  subdirectories = os.listdir(sub_folders[i])
  if len(subdirectories)==1:
     incomplete_directories.append(sub_folders[i])
complete_directories = [x for x in sub_folders if x not in incomplete_directories]

  ## Now we got the adress of the folders that have more than one folder named  "ses-1" in them.
  ## Now excluding the folders without insidescanner results
  ## I actually want to make a list of the folders that have psychometric files in them and later
  ## Will separate visual and auditory
desired_folder="InsideScanner"
sessions=["ses-2","ses-3"]
valuable_folders=[]
for session in sessions:
  for index in range(len(complete_directories)):
    reading_dir=os.path.join(complete_directories[index], session)
    reading_dir=os.path.join(reading_dir,desired_folder)
    if os.path.isdir(reading_dir) == False:
      continue
    valuable_folders.append(reading_dir)

## Now separating the vissual and auditory folders
visual_folders=[]
auditory_folders=[]
for current_folder in valuable_folders:
  contents=os.listdir(current_folder)
  for file in contents:
    if "Voddball" in file:
      visual_folders.append(current_folder)
      break
    elif "Aoddball" in file:
      auditory_folders.append(current_folder)
      break
dataholding_folders=auditory_folders+visual_folders
dataless_folders= [x for x in valuable_folders if x not in dataholding_folders]
## Now that the folders containing the files we need are found, we can start working on them to get the results we want
## auditory files


['/content/gdrive/Shareddrives/LC-Aging/Older Adult - MRI Study/Data Collection/BAP/BAP data/sub-BAP103', '/content/gdrive/Shareddrives/LC-Aging/Older Adult - MRI Study/Data Collection/BAP/BAP data/sub-BAP001', '/content/gdrive/Shareddrives/LC-Aging/Older Adult - MRI Study/Data Collection/BAP/BAP data/sub-BAP003', '/content/gdrive/Shareddrives/LC-Aging/Older Adult - MRI Study/Data Collection/BAP/BAP data/sub-BAP100', '/content/gdrive/Shareddrives/LC-Aging/Older Adult - MRI Study/Data Collection/BAP/BAP data/sub-BAP101', '/content/gdrive/Shareddrives/LC-Aging/Older Adult - MRI Study/Data Collection/BAP/BAP data/sub-BAP102', '/content/gdrive/Shareddrives/LC-Aging/Older Adult - MRI Study/Data Collection/BAP/BAP data/sub-BAP002', '/content/gdrive/Shareddrives/LC-Aging/Older Adult - MRI Study/Data Collection/BAP/BAP data/sub-BAP104', '/content/gdrive/Shareddrives/LC-Aging/Older Adult - MRI Study/Data Collection/BAP/BAP data/sub-BAP105', '/content/gdrive/Shareddrives/LC-Aging/Older Adult - M

In [ ]:
for scans_folder in auditory_folders:
  temp=substring_after(scans_folder,"sub-")
  subject_index=substring_before(temp,"/ses")
  print("Now reading subject %s" %(subject_index))
  scans_contents=os.listdir(scans_folder)
  mat_files= [element for element in scans_contents if element.endswith('.mat') and "eyetrack" not in element and "MVCnPRAC" not in element]
  for run_index in range(len(mat_files)):
    matfile_directory=os.path.join(scans_folder, mat_files[run_index])
    current_run=sio.loadmat(matfile_directory)
    isoddball=current_run['isoddball']
    iscorr=current_run['iscorr']
    trial_offset=current_run['trial_offset']
    reaction_time=current_run['Resp1EndTimeP_SBP']-current_run['Resp1StartTimeP']
    nan_indices=np.where(np.isnan(iscorr)==True)
    nan_indices=nan_indices[1]
    reaction_time=reaction_time[~np.isnan(iscorr)]
    log_reaction_time=np.log(reaction_time)
    zscore_reaction_time=zscore(log_reaction_time)
    RT_mean=np.mean(log_reaction_time)
    RT_std=np.std(log_reaction_time)
    up_std=RT_mean+3*RT_std
    down_std=RT_mean-3*RT_std
    iscorr_corrected=iscorr[~np.isnan(iscorr)]
    trial_offset_corrected=trial_offset[~np.isnan(iscorr)]
    isoddball_corrected=isoddball[~np.isnan(iscorr)]
    offsets_no_repeat_sorted=np.sort(np.unique(trial_offset))
    accuracy=np.zeros((np.size(offsets_no_repeat_sorted)))
    reaction=np.zeros((np.size(offsets_no_repeat_sorted)))
    corrects_vector=np.zeros((np.size(offsets_no_repeat_sorted)))
    trials_vector=np.zeros((np.size(offsets_no_repeat_sorted)))
    if run_index==0:
      accuracies=np.zeros((np.size(accuracy),len(mat_files)))
      reactions=np.zeros((np.size(reaction),len(mat_files)))
      conducted_trials=np.zeros((np.size(trials_vector),len(mat_files)))
      correct_trials=np.zeros((np.size(trials_vector),len(mat_files)))
    for i in range(len(offsets_no_repeat_sorted)):
      corrects=0
      total_time_to_react=0
      for j in range(len(iscorr_corrected)):
        if trial_offset_corrected[j]==offsets_no_repeat_sorted[i]:
          if iscorr_corrected[j]==1:
            corrects=corrects+1
          if log_reaction_time[j]>up_std:
            print("Outlier at index %d of run %d" %(j,run_index))
            continue
          elif log_reaction_time[j]<down_std:
            print("Outlier at index %d of run %d" %(j,run_index))
            continue
          else:
            total_time_to_react=total_time_to_react+zscore_reaction_time[j]
      accuracy[i]=corrects/np.count_nonzero(trial_offset_corrected==offsets_no_repeat_sorted[i])
      reaction[i]=total_time_to_react/np.count_nonzero(trial_offset_corrected==offsets_no_repeat_sorted[i])
      corrects_vector[i]=corrects
      trials_vector[i]=np.count_nonzero(trial_offset_corrected==offsets_no_repeat_sorted[i])
    accuracies[run_index,:]=accuracy
    reactions[run_index, :]=reaction
    conducted_trials[run_index,:]=trials_vector #3including only non-outliers
    correct_trials[run_index,:]=corrects_vector
    ## saving the auditory task results
    if path.exists('/content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/%s' %(subject_index)) == False:
      os.mkdir('/content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/%s' %(subject_index))
    if path.exists('/content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/%s/auditory' %(subject_index)) == False:
      os.mkdir('/content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/%s/auditory' %(subject_index))
    save_directory=('/content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/%s/auditory' %(subject_index))
    pd.DataFrame(accuracies).to_csv("%s/updated_auditory_accuracy.csv" %(save_directory))
    pd.DataFrame(reactions).to_csv("%s/updated_auditory_reaction_zscore.csv" %(save_directory))
    pd.DataFrame(conducted_trials).to_csv("%s/performed_trials.csv" %(save_directory))
    pd.DataFrame(correct_trials).to_csv("%s/corrects_number.csv" %(save_directory))


Now reading subject BAP001
Outlier at index 3 of run 4
Now reading subject BAP102
Now reading subject BAP104
Now reading subject BAP106
Outlier at index 11 of run 2
Now reading subject BAP108
Now reading subject BAP114
Outlier at index 11 of run 2
Outlier at index 22 of run 4
Now reading subject BAP116
Now reading subject BAP110
Now reading subject BAP120
Now reading subject BAP118
Now reading subject BAP122
Outlier at index 8 of run 1
Now reading subject BAP124
Now reading subject BAP146
Now reading subject BAP140
Now reading subject BAP132
Now reading subject BAP138
Now reading subject BAP134
Now reading subject BAP136
Now reading subject BAP128
Now reading subject BAP144
Now reading subject BAP103
Now reading subject BAP003
Outlier at index 8 of run 0
Now reading subject BAP101
Now reading subject BAP107
Now reading subject BAP109
Now reading subject BAP111
Now reading subject BAP117
Outlier at index 22 of run 1
Now reading subject BAP125
Outlier at index 11 of run 3
Now reading sub

In [7]:
for scans_folder in visual_folders:
  temp=substring_after(scans_folder,"sub-")
  subject_index=substring_before(temp,"/ses")
  print("Now reading subject %s" %(subject_index))
  scans_contents=os.listdir(scans_folder)
  mat_files= [element for element in scans_contents if element.endswith('.mat') and "eyetrack" not in element and "MVCnPRAC" not in element]
  for run_index in range(len(mat_files)):
    matfile_directory=os.path.join(scans_folder, mat_files[run_index])
    current_run=sio.loadmat(matfile_directory)
    isoddball=current_run['isoddball']
    iscorr=current_run['iscorr']
    trial_offset=current_run['trial_offset']
    reaction_time=current_run['Resp1EndTimeP_SBP']-current_run['Resp1StartTimeP']
    nan_indices=np.where(np.isnan(iscorr)==True)
    nan_indices=nan_indices[1]
    reaction_time=reaction_time[~np.isnan(iscorr)]
    log_reaction_time=np.log(reaction_time)
    zscore_reaction_time=zscore(log_reaction_time)
    RT_mean=np.mean(log_reaction_time)
    RT_std=np.std(log_reaction_time)
    up_std=RT_mean+3*RT_std
    down_std=RT_mean-3*RT_std
    iscorr_corrected=iscorr[~np.isnan(iscorr)]
    trial_offset_corrected=trial_offset[~np.isnan(iscorr)]
    isoddball_corrected=isoddball[~np.isnan(iscorr)]
    offsets_no_repeat_sorted=np.sort(np.unique(trial_offset))
    accuracy=np.zeros((np.size(offsets_no_repeat_sorted)))
    reaction=np.zeros((np.size(offsets_no_repeat_sorted)))
    corrects_vector=np.zeros((np.size(offsets_no_repeat_sorted)))
    trials_vector=np.zeros((np.size(offsets_no_repeat_sorted)))
    if run_index==0:
      accuracies=np.zeros((np.size(accuracy),len(mat_files)))
      reactions=np.zeros((np.size(reaction),len(mat_files)))
      conducted_trials=np.zeros((np.size(trials_vector),len(mat_files)))
      correct_trials=np.zeros((np.size(trials_vector),len(mat_files)))
    for i in range(len(offsets_no_repeat_sorted)):
      corrects=0
      total_time_to_react=0
      for j in range(len(iscorr_corrected)):
        if trial_offset_corrected[j]==offsets_no_repeat_sorted[i]:
          if iscorr_corrected[j]==1:
            corrects=corrects+1
          if log_reaction_time[j]>up_std:
            print("Outlier at index %d of run %d" %(j,run_index))
            continue
          elif log_reaction_time[j]<down_std:
            print("Outlier at index %d of run %d" %(j,run_index))
            continue
          else:
            total_time_to_react=total_time_to_react+zscore_reaction_time[j]
      accuracy[i]=corrects/np.count_nonzero(trial_offset_corrected==offsets_no_repeat_sorted[i])
      reaction[i]=total_time_to_react/np.count_nonzero(trial_offset_corrected==offsets_no_repeat_sorted[i])
      corrects_vector[i]=corrects
      trials_vector[i]=np.count_nonzero(trial_offset_corrected==offsets_no_repeat_sorted[i])
    accuracies[run_index,:]=accuracy
    reactions[run_index, :]=reaction
    conducted_trials[run_index,:]=trials_vector #3including only non-outliers
    correct_trials[run_index,:]=corrects_vector
    ## saving the visual task results
    if path.exists('/content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/%s' %(subject_index)) == False:
      os.mkdir('/content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/%s' %(subject_index))
    if path.exists('/content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/%s/visual' %(subject_index)) == False:
      os.mkdir('/content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/%s/visual' %(subject_index))
    save_directory=('/content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/%s/visual' %(subject_index))
    pd.DataFrame(accuracies).to_csv("%s/updated_visual_accuracy.csv" %(save_directory))
    pd.DataFrame(reactions).to_csv("%s/updated_visual_reaction_zscore.csv" %(save_directory))
    pd.DataFrame(conducted_trials).to_csv("%s/performed_trials.csv" %(save_directory))
    pd.DataFrame(correct_trials).to_csv("%s/corrects_number.csv" %(save_directory))

Now reading subject BAP103
Now reading subject BAP003
Now reading subject BAP101
Now reading subject BAP107
Now reading subject BAP109
Outlier at index 19 of run 3
Now reading subject BAP111
Outlier at index 22 of run 4
Now reading subject BAP117
Outlier at index 22 of run 2
Now reading subject BAP125
Now reading subject BAP121
Now reading subject BAP123
Now reading subject BAP133
Now reading subject BAP147
Now reading subject BAP145
Now reading subject BAP139
Outlier at index 4 of run 0
Outlier at index 8 of run 3
Now reading subject BAP135
Outlier at index 20 of run 1
Outlier at index 0 of run 2
Outlier at index 18 of run 3
Now reading subject BAP129
Now reading subject BAP127
Outlier at index 0 of run 0
Outlier at index 7 of run 2
Outlier at index 11 of run 3
Outlier at index 17 of run 4
Now reading subject BAP151
Outlier at index 9 of run 3
Now reading subject BAP157
Now reading subject BAP001
Now reading subject BAP102
Now reading subject BAP104
Outlier at index 20 of run 4
Now re

In [5]:
print(subject_index)

BAP149


In [ ]:
save_directory

NameError: ignored